In [2]:
import pandas as pd
import pandas_profiling as pds
import pandas_profiling as ProfileReport
import glob
import os, fnmatch
import sys
from functools import reduce
import re


Matplotlib is building the font cache; this may take a moment.


In [3]:
def get_file(directory, filetype, filenumber):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    listOfFiles = os.listdir(directory)
    pattern = f"*{filetype}"
    df_list = []
    csv_files = [] 
    for entry in listOfFiles:
        if fnmatch.fnmatch(entry, pattern):
            csv_files.append(entry)
            df_list.append((directory + entry))
    filename = [csv.replace(filetype, '') for csv in csv_files] #replace each delimiter in turn with a space
    series = [pd.read_csv(df) for df in df_list]
#     print (entry)
#     print (filename)
    return series[filenumber]  # Self-explanatory.

In [4]:
circuits_raw_df = get_file('../data/raw/', '.csv', 0)
constructors_raw_df = get_file('../data/raw/', '.csv', 1)
constructor_results_raw_df = get_file('../data/raw/', '.csv', 2)
constructor_standings_raw_df = get_file('../data/raw/', '.csv', 3)
drivers_raw_df = get_file('../data/raw/', '.csv', 4)
driver_standings_raw_df = get_file('../data/raw/', '.csv', 5)
lap_times_raw_df = get_file('../data/raw/', '.csv', 6)
pit_stops_raw_df = get_file('../data/raw/', '.csv', 7)
qualifying_raw_df = get_file('../data/raw/', '.csv', 8)
races_raw_df = get_file('../data/raw/', '.csv', 9)
results_raw_df = get_file('../data/raw/', '.csv', 10)
seasons_raw_df = get_file('../data/raw/', '.csv', 11)
status_raw_df = get_file('../data/raw/', '.csv', 12)

In [5]:
# I can see time will be a problem, I will set all the times to seconds for all dataframes with time intervals: 
def hh_mm_ss2seconds(hh_mm_ss):
    return reduce(lambda acc, x: acc*60 + x, map(float, hh_mm_ss.split(':')))


In [19]:
circuits_raw_df

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,\N,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,\N,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,\N,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,\N,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
71,72,port_imperial,Port Imperial Street Circuit,New Jersey,USA,40.77690,-74.01110,\N,http://en.wikipedia.org/wiki/Port_Imperial_Str...
72,73,BAK,Baku City Circuit,Baku,Azerbaijan,40.37250,49.85330,\N,http://en.wikipedia.org/wiki/Baku_City_Circuit
73,74,hanoi,Hanoi Street Circuit,Hanoi,Vietnam,21.01660,105.76600,\N,http://en.wikipedia.org/wiki/Hanoi_Street_Circuit
74,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,\N,http://en.wikipedia.org/wiki/Algarve_Internati...


In [20]:
# Alt value no consistent information
circuits_interim_df = pd.read_csv('../data/raw/circuits.csv').drop(['alt'], axis=1)
circuits_interim_df.to_csv('../data/interim/i_circuits.csv', index=False)

In [21]:
constructors_raw_df

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
...,...,...,...,...,...
206,209,manor,Manor Marussia,British,http://en.wikipedia.org/wiki/Manor_Motorsport
207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team
209,212,alpha_tauri,Scuderia Alpha Tauri,Italian,http://en.wikipedia.org/wiki/Scuderia_Alpha_Tauri


In [22]:
# no changes made so I filename was not modified
constructors_interim_df = pd.read_csv('../data/raw/constructors.csv')
constructors_interim_df.to_csv('../data/interim/constructors.csv', index=False)

In [23]:
constructor_results_raw_df

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,\N
1,2,18,2,8.0,\N
2,3,18,3,9.0,\N
3,4,18,4,5.0,\N
4,5,18,5,2.0,\N
...,...,...,...,...,...
11675,16175,1042,211,6.0,\N
11676,16176,1042,4,6.0,\N
11677,16177,1042,51,0.0,\N
11678,16178,1042,3,0.0,\N


In [24]:
# status value no information just \N
constructor_results_interim_df = pd.read_csv('../data/raw/constructor_results.csv').drop(['status'], axis=1)
constructor_results_interim_df.to_csv('../data/interim/i_constructor_results.csv', index=False)

In [6]:
drivers_raw_df
drivers_raw_df['full name'] = drivers_raw_df[['forename','surname']].apply(lambda x: ' '.join(x), axis=1)

In [7]:
drivers_raw_df.drop(columns= ['forename', 'surname', 'url'], inplace = True)
drivers_raw_df.to_csv('../data/processed/driver.csv')

In [8]:
drivers_raw_df

,driverId,driverRef,number,code,dob,nationality,full name
0,1,hamilton,44,HAM,1985-01-07,British,Lewis Hamilton
1,2,heidfeld,\N,HEI,1977-05-10,German,Nick Heidfeld
2,3,rosberg,6,ROS,1985-06-27,German,Nico Rosberg
3,4,alonso,14,ALO,1981-07-29,Spanish,Fernando Alonso
4,5,kovalainen,\N,KOV,1981-10-19,Finnish,Heikki Kovalainen
...,...,...,...,...,...,...,...
843,845,sirotkin,35,SIR,1995-08-27,Russian,Sergey Sirotkin
844,846,norris,4,NOR,1999-11-13,British,Lando Norris
845,847,russell,63,RUS,1998-02-15,British,George Russell
846,848,albon,23,ALB,1996-03-23,Thai,Alexander Albon


In [31]:
rank_winners_2020 = ['Lewis Hamilton', 'Valtteri Bottas', 'Max Verstappen', 'Sergio Pérez', 'Charles Leclerc', 'Daniel Ricciardo', 'Carlos Sainz', 'Alexander Albon', 'Lando Norris', 'Pierre Gasly']
# driver = [name for name in winners_2019 ] 
# [row for row in drivers_raw_df['driverRef'] if  driver[0] in row]

In [32]:
def name_check(stringy):
    return stringy in rank_winners_2020

In [33]:
rank_winners_2020 = drivers_raw_df[drivers_raw_df['full name'].map(name_check)]
rank_winners_2020['driverId']

0        1
452    842
814    815
816    817
821    822
829    830
831    832
842    844
844    846
846    848
Name: driverId, dtype: int64

In [34]:
rank_winners_2020

,driverId,driverRef,number,code,dob,nationality,url,full name
0,1,hamilton,44,HAM,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis Hamilton
452,842,gasly,10,GAS,1996-02-07,French,http://en.wikipedia.org/wiki/Pierre_Gasly,Pierre Gasly
814,815,perez,11,PER,1990-01-26,Mexican,http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez,Sergio Pérez
816,817,ricciardo,3,RIC,1989-07-01,Australian,http://en.wikipedia.org/wiki/Daniel_Ricciardo,Daniel Ricciardo
821,822,bottas,77,BOT,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas,Valtteri Bottas
829,830,max_verstappen,33,VER,1997-09-30,Dutch,http://en.wikipedia.org/wiki/Max_Verstappen,Max Verstappen
831,832,sainz,55,SAI,1994-09-01,Spanish,http://en.wikipedia.org/wiki/Carlos_Sainz_Jr.,Carlos Sainz
842,844,leclerc,16,LEC,1997-10-16,Monegasque,http://en.wikipedia.org/wiki/Charles_Leclerc,Charles Leclerc
844,846,norris,4,NOR,1999-11-13,British,http://en.wikipedia.org/wiki/Lando_Norris,Lando Norris
846,848,albon,23,ALB,1996-03-23,Thai,http://en.wikipedia.org/wiki/Alexander_Albon,Alexander Albon


In [35]:
# 'hamilton' in ['hamilton', 'bottas', 'verstappen', 'leclerc', 'vettel', 'sainz', 'gasly', 'albon', 'ricciardo', 'perez']

In [318]:
# no changes made so I filename was not modified, want to wait to match relevant datasets before dropping \N
drivers_interim_df = pd.read_csv('../data/raw/drivers.csv')
drivers_interim_df.to_csv('../data/interim/drivers.csv', index=False)

In [13]:
driver_standings_raw_df

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0
...,...,...,...,...,...,...,...
32810,70073,1042,825,1.0,19,19,0
32811,70074,1042,840,57.0,11,11,0
32812,70075,1042,817,80.0,4,4,0
32813,70076,1042,830,162.0,3,3,1


In [14]:
# no changes made so I filename was not modified, want to wait to match relevant datasets before dropping \N
driver_standings_interim_df = pd.read_csv('../data/raw/driver_standings.csv')
driver_standings_interim_df.to_csv('../data/interim/driver_standings.csv', index=False)

In [15]:
lap_times_raw_df

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342
...,...,...,...,...,...,...
485105,1042,815,61,5,1:21.164,81164
485106,1042,815,62,5,1:21.427,81427
485107,1042,815,63,5,1:21.925,81925
485108,1042,815,64,6,1:23.139,83139


In [16]:
lap_times_interim_df = pd.read_csv('../data/raw/lap_times.csv',
                     converters={'time': hh_mm_ss2seconds})
lap_times_interim_df

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,98.109,98109
1,841,20,2,1,93.006,93006
2,841,20,3,1,92.713,92713
3,841,20,4,1,92.803,92803
4,841,20,5,1,92.342,92342
...,...,...,...,...,...,...
485105,1042,815,61,5,81.164,81164
485106,1042,815,62,5,81.427,81427
485107,1042,815,63,5,81.925,81925
485108,1042,815,64,6,83.139,83139


In [17]:
# changes made to time to be in seconds as float
lap_times_interim_df.to_csv('../data/interim/i_lap_times.csv', index=False)

In [18]:
pit_stops_raw_df

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842
...,...,...,...,...,...,...,...
7838,1042,846,2,43,14:13:48,27.750,27750
7839,1042,815,2,45,14:15:49,26.635,26635
7840,1042,848,2,46,14:17:42,25.882,25882
7841,1042,839,1,53,14:26:47,28.535,28535


In [19]:
pit_stops_interim_df = pd.read_csv('../data/raw/pit_stops.csv',
                     converters={'time': hh_mm_ss2seconds})
pit_stops_interim_df

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,61523.0,26.898,26898
1,841,30,1,1,61552.0,25.021,25021
2,841,17,1,11,62448.0,23.426,23426
3,841,4,1,12,62554.0,23.251,23251
4,841,13,1,13,62650.0,23.842,23842
...,...,...,...,...,...,...,...
7838,1042,846,2,43,51228.0,27.750,27750
7839,1042,815,2,45,51349.0,26.635,26635
7840,1042,848,2,46,51462.0,25.882,25882
7841,1042,839,1,53,52007.0,28.535,28535


In [20]:
# changes made to time to be in seconds as float
pit_stops_interim_df.to_csv('../data/interim/i_pit_stops.csv', index=False)

In [21]:
qualifying_raw_df

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
8589,8630,1042,8,51,7,16,1:18.201,NaN,NaN
8590,8631,1042,841,51,99,17,1:18.323,NaN,NaN
8591,8632,1042,154,210,8,18,1:18.364,NaN,NaN
8592,8633,1042,825,210,20,19,1:18.508,NaN,NaN


In [22]:
# replace all null values with zeroes for time conversion: 
qualifying_raw_df['q1'] = qualifying_raw_df['q1'].fillna(0)
qualifying_raw_df['q2'] = qualifying_raw_df['q2'].fillna(0)
qualifying_raw_df['q3'] = qualifying_raw_df['q3'].fillna(0)

In [23]:
qualifying_raw = qualifying_raw_df.replace({'q1': '\\N', 'q2': '\\N', 'q3': '\\N'}, 0)

In [24]:
qualifying_raw.to_csv('../data/interim/N_A_qualifying.csv', index=False)

In [25]:
qualifying_interim_df = pd.read_csv('../data/interim/N_A_qualifying.csv',
                     converters={'q1': hh_mm_ss2seconds, 'q2': hh_mm_ss2seconds, 'q3': hh_mm_ss2seconds})
qualifying_interim_df

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,86.572,85.187,86.714
1,2,18,9,2,4,2,86.103,85.315,86.869
2,3,18,5,1,23,3,85.664,85.452,87.079
3,4,18,13,6,2,4,85.994,85.691,87.178
4,5,18,2,2,3,5,85.960,85.518,87.236
...,...,...,...,...,...,...,...,...,...
8589,8630,1042,8,51,7,16,78.201,0.000,0.000
8590,8631,1042,841,51,99,17,78.323,0.000,0.000
8591,8632,1042,154,210,8,18,78.364,0.000,0.000
8592,8633,1042,825,210,20,19,78.508,0.000,0.000


In [26]:
#Had to remove null values and \\N
qualifying_interim_df.to_csv('../data/interim/i_qualifying.csv', index=False)

In [27]:
races_raw_df

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
...,...,...,...,...,...,...,...,...
1030,1043,2020,13,21,Emilia Romagna Grand Prix,2020-11-01,12:10:00,https://en.wikipedia.org/wiki/2020_Emilia_Roma...
1031,1044,2020,14,5,Turkish Grand Prix,2020-11-15,10:10:00,https://en.wikipedia.org/wiki/2020_Turkish_Gra...
1032,1045,2020,15,3,Bahrain Grand Prix,2020-11-29,15:10:00,https://en.wikipedia.org/wiki/2020_Bahrain_Gra...
1033,1046,2020,16,3,Sakhir Grand Prix,2020-12-06,17:10:00,https://en.wikipedia.org/wiki/2020_Sakhir_Gran...


In [28]:
# Just in case
races_raw = races_raw_df.replace({'time': '\\N'}, 0)
races_raw.to_csv('../data/interim/N_A_races.csv', index=False)

In [29]:
races_interim_df = pd.read_csv('../data/interim/N_A_races.csv',
                     converters={'time': hh_mm_ss2seconds})
races_interim_df

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,21600.0,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,32400.0,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,25200.0,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,43200.0,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,43200.0,http://en.wikipedia.org/wiki/2009_Spanish_Gran...
...,...,...,...,...,...,...,...,...
1030,1043,2020,13,21,Emilia Romagna Grand Prix,2020-11-01,43800.0,https://en.wikipedia.org/wiki/2020_Emilia_Roma...
1031,1044,2020,14,5,Turkish Grand Prix,2020-11-15,36600.0,https://en.wikipedia.org/wiki/2020_Turkish_Gra...
1032,1045,2020,15,3,Bahrain Grand Prix,2020-11-29,54600.0,https://en.wikipedia.org/wiki/2020_Bahrain_Gra...
1033,1046,2020,16,3,Sakhir Grand Prix,2020-12-06,61800.0,https://en.wikipedia.org/wiki/2020_Sakhir_Gran...


In [36]:
results_raw_df

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24855,24861,1042,825,210,20,19,16,16,16,0.0,65,\N,\N,63,16,1:21.460,205.632,11
24856,24862,1042,154,210,8,18,17,17,17,0.0,65,\N,\N,59,17,1:21.664,205.118,11
24857,24863,1042,849,3,6,20,18,18,18,0.0,64,\N,\N,53,19,1:21.859,204.629,12
24858,24864,1042,826,213,26,13,19,19,19,0.0,64,\N,\N,59,8,1:20.449,208.216,12


In [37]:
results_raw_df['position'].value_counts()

\N    10721
4      1040
3      1040
2      1038
5      1036
1      1033
6      1028
7      1009
8       981
9       943
10      883
11      806
12      705
13      618
14      511
15      438
16      352
17      261
18      170
19      113
20       61
21       34
22       19
23        8
24        3
26        1
28        1
33        1
30        1
32        1
27        1
31        1
25        1
29        1
Name: position, dtype: int64

In [38]:
results_raw_df["time"].value_counts()

\N           18329
+8:22.19         5
+1:29.6          4
+46.2            4
+0.7             4
             ...  
+2:33.4          1
+1:23.8          1
+14.5            1
+1:07.970        1
+22.569          1
Name: time, Length: 6306, dtype: int64

In [39]:
results_raw_df['Win or Lost'] = results_raw_df['time']

In [40]:
results_raw_df['time'] = results_raw_df['time'].str.replace('+', '')
results_raw_df['time']

0        1:34:50.616
1              5.478
2              8.163
3             17.181
4             18.014
            ...     
24855             \N
24856             \N
24857             \N
24858             \N
24859             \N
Name: time, Length: 24860, dtype: object

In [41]:
results = results_raw_df.replace({'fastestLapTime': '\\N', 'time': '\\N', 'milliseconds': '\\N', 'fastestLapSpeed': '\\N', 'Win or Lost': '\\N', 'fastestLap': '\\N'}, 0)

In [42]:
results['milliseconds'].dtypes

dtype('O')

In [43]:
results['milliseconds'] = pd.to_numeric(results['milliseconds'], downcast="float")
results['fastestLap'] = pd.to_numeric(results['fastestLap'])

In [44]:
results['milliseconds'] = results['milliseconds'].div(1000)

In [45]:
results.to_csv('../data/interim/N_A_results.csv', index=False)

In [46]:
results_interim_df = pd.read_csv('../data/interim/N_A_results.csv',
                     converters={'fastestLapTime': hh_mm_ss2seconds})
results_interim_df

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,Win or Lost
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690.616,39,2,87.452,218.300,1,1:34:50.616
1,2,18,2,2,3,5,2,2,2,8.0,58,5.478,5696.094,41,3,87.739,217.586,1,+5.478
2,3,18,3,3,7,7,3,3,3,6.0,58,8.163,5698.779,41,5,88.090,216.719,1,+8.163
3,4,18,4,4,5,11,4,4,4,5.0,58,17.181,5707.797,58,7,88.603,215.464,1,+17.181
4,5,18,5,1,23,3,5,5,5,4.0,58,18.014,5708.630,43,1,87.418,218.385,1,+18.014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24855,24861,1042,825,210,20,19,16,16,16,0.0,65,0,0.000,63,16,81.460,205.632,11,0
24856,24862,1042,154,210,8,18,17,17,17,0.0,65,0,0.000,59,17,81.664,205.118,11,0
24857,24863,1042,849,3,6,20,18,18,18,0.0,64,0,0.000,53,19,81.859,204.629,12,0
24858,24864,1042,826,213,26,13,19,19,19,0.0,64,0,0.000,59,8,80.449,208.216,12,0


In [47]:
results_interim_df.loc[(results_interim_df["Win or Lost"].str.match("\+\d+") | results_interim_df["Win or Lost"].str.match('0')) == True, 'Win or Lost'] = "lost"

In [48]:
results_interim_df.loc[(results_interim_df["Win or Lost"].str.match("lost") == False, 'Win or Lost')] = "win"

In [49]:
wl_results = results_interim_df.replace({'Win or Lost': '0'}, 'lost')
wl_results['Win or Lost'].value_counts()

lost    23825
win      1035
Name: Win or Lost, dtype: int64

In [50]:
s = wl_results.groupby('driverId')['Win or Lost']
counts = s.value_counts()
percent = s.value_counts(normalize=True)
# percent100 = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
percent100 = s.value_counts(normalize=True).mul(100).round(1)
wl_ratio = pd.DataFrame({'counts': counts, 'per': percent, 'per100': percent100})
wl_ratio.reset_index(inplace= True)
wl_ratio

,driverId,Win or Lost,counts,per,per100
0,1,lost,170,0.648855,64.9
1,1,win,92,0.351145,35.1
2,2,lost,184,1.000000,100.0
3,3,lost,183,0.888350,88.8
4,3,win,23,0.111650,11.2
...,...,...,...,...,...
951,845,lost,21,1.000000,100.0
952,846,lost,33,1.000000,100.0
953,847,lost,33,1.000000,100.0
954,848,lost,33,1.000000,100.0


In [51]:
wl_results.to_csv('../data/processed/wl_results.csv', index=False)

In [52]:
fastest_lap_time = wl_results.groupby('driverId')['fastestLapTime'].agg(['min','max','mean'])
fastest_lap_time.rename(columns={'min':'Slowest Lap Time', 'max': 'Fastest Lap Time', 'mean': 'Average Lap Time'}).reset_index()

,driverId,Slowest Lap Time,Fastest Lap Time,Average Lap Time
0,1,0.000,119.560,89.571771
1,2,0.000,114.158,55.679641
2,3,0.000,170.950,89.036680
3,4,0.000,133.579,77.726134
4,5,0.000,130.404,91.033268
...,...,...,...,...
843,845,0.000,109.113,85.576333
844,846,67.193,108.552,88.388000
845,847,68.601,111.754,88.958091
846,848,67.299,108.736,87.662303


In [53]:
speed = wl_results.groupby('driverId')['fastestLapSpeed'].agg(['min','max','mean'])
speed.rename(columns={'min':'Minimum Speed', 'max': 'Maximum Speed', 'mean': 'Average Speed'})

,Minimum Speed,Maximum Speed,Average Speed
driverId,,,
1,0.000,255.014,201.224538
2,0.000,250.375,128.636799
3,0.000,246.884,192.512102
4,0.000,253.874,174.328226
5,0.000,247.605,192.936000
...,...,...,...
845,0.000,246.568,195.894143
846,157.214,248.141,212.342030
847,155.938,247.033,210.987364


In [54]:
lap_speed = wl_results.groupby('driverId')['fastestLap'].agg(['min','max','mean'])
lap_speed.rename(columns={'min':'Minimum Laps', 'max': 'Maximum Laps', 'mean': 'Average Laps'}).reset_index()

,driverId,Minimum Laps,Maximum Laps,Average Laps
0,1,0,76,43.332061
1,2,0,77,24.788043
2,3,0,76,42.218447
3,4,0,77,36.363057
4,5,0,76,43.339286
...,...,...,...,...
843,845,0,75,45.190476
844,846,4,71,45.272727
845,847,5,76,46.363636
846,848,9,68,48.030303


In [55]:
wl_ratio.to_csv('../data/processed/top10_wl.csv', index=False)
wl_ratio

,driverId,Win or Lost,counts,per,per100
0,1,lost,170,0.648855,64.9
1,1,win,92,0.351145,35.1
2,2,lost,184,1.000000,100.0
3,3,lost,183,0.888350,88.8
4,3,win,23,0.111650,11.2
...,...,...,...,...,...
951,845,lost,21,1.000000,100.0
952,846,lost,33,1.000000,100.0
953,847,lost,33,1.000000,100.0
954,848,lost,33,1.000000,100.0


In [427]:
top10_rank_stats = rank_winners_2020.merge(wl_ratio, on = 'driverId', how = 'left')
top10_rank_stats.drop(columns='url', inplace=True)
top10_rank_stats.to_csv('../data/processed/top10_results.csv', index=False)

In [428]:
speed_index = rank_winners_2020.merge(fastest_lap_time, on = 'driverId', how = 'left').merge(speed, on='driverId', how='left').merge(lap_speed, on='driverId', how='left').fillna(0)
speed_index.drop(columns='url', inplace=True)
speed_index.rename(columns={'min_x':'Slowest Lap Time', 'man_x':'Fastest Lap Time', 'mean_x': 'Average Lap Time', 'min_y': 'Minimum Speed', 'max_y': 'Maximum Speed', 'mean_y': 'Average Speed', 'min': 'Minimum Laps', 'max': 'Maximum Laps', 'mean': 'Average Laps'}, inplace = True)
speed_index

,driverId,driverRef,number,code,dob,nationality,full name,Slowest Lap Time,max_x,Average Lap Time,Minimum Speed,Maximum Speed,Average Speed,Minimum Laps,Maximum Laps,Average Laps
0,1,hamilton,44,HAM,1985-01-07,British,Lewis Hamilton,0.000,119.560,89.571771,0.000,255.014,201.224538,0,76,43.332061
1,842,gasly,10,GAS,1996-02-07,French,Pierre Gasly,0.000,109.700,84.602356,0.000,248.611,198.936186,0,72,45.915254
2,817,ricciardo,3,RIC,1989-07-01,Australian,Daniel Ricciardo,0.000,119.831,90.965011,0.000,250.174,199.633481,0,78,45.360656
3,822,bottas,77,BOT,1989-08-28,Finnish,Valtteri Bottas,0.000,151.939,90.828822,0.000,251.690,203.473487,0,74,47.019737
4,830,max_verstappen,33,VER,1997-09-30,Dutch,Max Verstappen,0.000,140.776,85.783746,0.000,250.830,195.509430,0,70,43.035088
5,832,sainz,55,SAI,1994-09-01,Spanish,Carlos Sainz,0.000,134.103,84.740167,0.000,249.671,188.966746,0,70,41.035088
6,844,leclerc,16,LEC,1997-10-16,Monegasque,Charles Leclerc,0.000,107.840,85.467019,0.000,251.235,202.433019,0,70,43.259259
7,846,norris,4,NOR,1999-11-13,British,Lando Norris,67.193,108.552,88.388000,157.214,248.141,212.342030,4,71,45.272727
8,848,albon,23,ALB,1996-03-23,Thai,Alexander Albon,67.299,108.736,87.662303,158.890,250.165,214.058455,9,68,48.030303


In [429]:
speed_index.to_csv('../data/processed/top10_speed_results.csv', index=False)

In [430]:
seasons_raw_df

,year,url
0,2009,https://en.wikipedia.org/wiki/2009_Formula_One...
1,2008,https://en.wikipedia.org/wiki/2008_Formula_One...
2,2007,https://en.wikipedia.org/wiki/2007_Formula_One...
3,2006,https://en.wikipedia.org/wiki/2006_Formula_One...
4,2005,https://en.wikipedia.org/wiki/2005_Formula_One...
...,...,...
66,2016,https://en.wikipedia.org/wiki/2016_Formula_One...
67,2017,https://en.wikipedia.org/wiki/2017_Formula_One...
68,2018,https://en.wikipedia.org/wiki/2018_Formula_One...
69,2019,https://en.wikipedia.org/wiki/2019_Formula_One...


In [22]:
races_interim_df.iloc['name', ]

In [22]:
# no changes made so I filename was not modified, want to wait to match relevant datasets before dropping \N
seasons_interim_df = pd.read_csv('../data/raw/seasons.csv')
seasons_interim_df.to_csv('../data/interim/seasons.csv', index=False)

In [25]:
status_raw_df

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine
...,...,...
131,134,+38 Laps
132,135,Brake duct
133,136,Seat
134,137,Damage


In [26]:
# no changes made so I filename was not modified, want to wait to match relevant datasets before dropping \N
status_interim_df = pd.read_csv('../data/raw/status.csv')
status_interim_df.to_csv('../data/interim/status.csv', index=False)

In [ ]:
The number of Grand prix occurrences between (2010–2020)? 
What are the top 